## PORT MIS_울산 전처리

### 0. 데이터 불러오기

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

# 데이터 파일이 저장된 디렉토리 경로
directory = '../data/울산'

# 데이터프레임을 저장할 리스트 생성
dfs = []

# 202001부터 202212까지의 월을 대상으로 반복문 실행
for year in range(2020, 2023):
    for month in range(1, 13):
        # 파일 경로 생성
        file_path = f'{directory}/울산_시설_{year}{month:02d}.xlsx'
        
        # 엑셀 파일을 읽어와 데이터프레임 생성 후 리스트에 추가
        try:
            df = pd.read_excel(file_path)
            dfs.append(df)
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {file_path}")

# 모든 데이터프레임을 하나로 합치기
result_df = pd.concat(dfs, ignore_index=True)

# 결과를 출력하거나 추가적인 작업을 수행할 수 있습니다.
print(result_df)


       시설사용허가현황 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5  \
0           NaN        NaN        NaN        NaN        NaN        NaN   
1           NaN        NaN        NaN        NaN        NaN        NaN   
2            순번       호출부호       입항횟수       입항횟수       입항횟수       신고톤수   
3             1     184803       2020        001        001      1.995   
4             2       DSMB       2020        001        001      5.025   
...         ...        ...        ...        ...        ...        ...   
148643     4109     210043       2022        118        002        499   
148644     4110     210043       2022        119        001        499   
148645     4111     210043       2022        119        002        499   
148646     4112     210043       2022        120        001        499   
148647     4113     210043       2022        120        002        499   

       Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9  ...       Unnamed: 12  \
0             NaN        NaN      

In [3]:
combined_df = result_df.copy()

In [4]:
import pandas as pd

directory = '../data/울산'

# List to store DataFrames
dfs = []

# Loop through the years 2020 to 2022
for year in range(2020, 2023):
    file_path_ship = f'{directory}/울산_입출항_{year}.xlsx'
    
    # Read Excel file and append DataFrame to the list
    try:
        df_ship = pd.read_excel(file_path_ship)
        dfs.append(df_ship)
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path_ship}")

# Concatenate the DataFrames
dfs = pd.concat(dfs, ignore_index=True)


In [5]:
dfs = dfs.iloc[11:]  # 상단 11행 삭제 

In [6]:
dfs.columns

Index(['선박입출항현황', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27'],
      dtype='object')

In [7]:
#dfs.drop(columns = 'z', inplace = True)

In [8]:
dfs.columns = ['항명', '호출부호', '선박명', '입항횟수_연도', '입항횟수_횟수', '구분', '외내', '입출', '총톤수', '입항일시',
       '출항일시', 'CIQ수속일자', '수리일시', '항해구분', 'MRN 번호', '계선장소_코드', '계선장소_숫자', '계선장소명',
       '차항지', '전출항지', '선박용도', '외항:한국인선원수/내항:해기사선원수', '외항:외국인선원수/내항:보통선원수',
       '승객', '예선', '도선', '부선호출부호1', '부선호출부호2']

In [9]:
combined_df.columns = ['순번', '호출부호', '입항횟수_연도', '입항횟수_횟수', '시설사용횟수', '신고톤수', '선박명', '선사/대리점_코드',
       '선사/대리점명', '신청시설_코드', '신청시설_번호', '신청시설명', '신청일시(FROM)',
       '신청일시(TO)', '지정시설_코드', '지정시설_번호', '지정시설명', '지정일시(FROM)',
       '지정일시(TO)', '사용목적명', '예보일시', '허가유무']

In [10]:
combined_df = combined_df.iloc[3:]

In [11]:
combined_df.to_excel(directory + '/울산_시설사용허가_202122.xlsx', index=False)

### 1. 컬럼 제거 및 결측치 확인

#### 컬럼 제거(시설허가 : combined_df , 입출항 : dfs)

In [12]:
columns_to_drop = ['신청시설_코드', '신청시설_번호', '신청시설명', '신청일시(FROM)', '신청일시(TO)', '선사/대리점_코드', '선사/대리점명']
combined_df.drop(columns=columns_to_drop, inplace=True)

In [13]:
columns_to_drop_dfs = ['외내', 'CIQ수속일자', 'MRN 번호', '차항지', '전출항지','외항:한국인선원수/내항:해기사선원수','외항:외국인선원수/내항:보통선원수', '승객', '예선', '도선', '부선호출부호1', '부선호출부호2']
dfs.drop(columns=columns_to_drop_dfs,inplace=True)

#### 상세처리

- 입출 : 출항만 사용
- 선박용도 > 최종 전처리시 많은 순으로 고려
- 미허가 행 제외

In [14]:
combined_df = combined_df[combined_df['허가유무'] != '미허가']

In [15]:
dfs = dfs[dfs['입출'] == '출항']

### 2. Merge_type 변경 후 Merge

In [16]:
combined_df['입항횟수_횟수'] = pd.to_numeric(combined_df['입항횟수_횟수'], errors='coerce')
combined_df['입항횟수_연도'] = pd.to_numeric(combined_df['입항횟수_연도'], errors='coerce')

dfs['입항횟수_횟수'] = dfs['입항횟수_횟수'].astype(float)
dfs['입항횟수_연도'] = dfs['입항횟수_연도'].astype(float)
combined_df['입항횟수_횟수'] = combined_df['입항횟수_횟수'].astype(float)
combined_df['입항횟수_연도'] = combined_df['입항횟수_연도'].astype(float)


In [17]:
merged_left = pd.merge(dfs, combined_df, on= ['호출부호','선박명','입항횟수_횟수','입항횟수_연도'], how='left') # 입출항, 시설사용허가 left merge
merged_left

,항명,호출부호,선박명,입항횟수_연도,입항횟수_횟수,구분,입출,총톤수,입항일시,출항일시,...,시설사용횟수,신고톤수,지정시설_코드,지정시설_번호,지정시설명,지정일시(FROM),지정일시(TO),사용목적명,예보일시,허가유무
0,울산,3EFF5,EVER CHARM,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-06-16 20:05,...,001,17.933,MQM,01,현대미포의장안벽 01,2020-06-16 13:40,2020-06-16 13:50,기타,2020-06-17,허가완료
1,울산,3EFF5,EVER CHARM,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-06-16 20:05,...,002,17.933,WAB,02,(구)벙커링전용정박지 E2,2020-06-16 14:52,2020-06-16 20:05,급유,2020-06-17,허가완료
2,울산,3ETC7,EVER CHEER,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-07-07 19:42,...,001,17.933,MQM,04,현대미포의장안벽 04,2020-07-07 13:40,2020-07-07 13:50,기타,2020-07-09,허가완료
3,울산,3ETC7,EVER CHEER,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-07-07 19:42,...,002,17.933,WAB,02,(구)벙커링전용정박지 E2,2020-07-07 14:48,2020-07-07 19:42,급유,2020-07-09,허가완료
4,울산,3FAW9,ESTEEM HOUSTON,2020.0,1.0,최종,출항,"29,651",1950-01-01 00:00,2020-06-07 15:36,...,001,29.651,MQM,03,현대미포의장안벽 03,2020-06-05 16:20,2020-06-05 16:30,접안대기,2020-06-08,허가완료
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149800,울산,130064,에이원,2022.0,114.0,최종,출항,431,2022-12-31 09:45,2022-12-31 16:15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149801,울산,HOXG,SPRING NOTE,2022.0,31.0,최종,출항,"2,579",2022-12-31 10:05,2022-12-31 20:10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149802,울산,200013,유성,2022.0,142.0,최종,출항,"1,998",2022-12-31 15:55,2022-12-31 23:15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149803,울산,066597,한성8호,2022.0,22.0,최종,출항,164,2022-12-31 17:55,2022-12-31 20:40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 중복된 행 제거

In [18]:
# 중복된 행, unique값만 남기기
unique_values = merged_left.drop_duplicates()

In [19]:
unique_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145548 entries, 0 to 149804
Data columns (total 27 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   항명          145548 non-null  object 
 1   호출부호        145548 non-null  object 
 2   선박명         145548 non-null  object 
 3   입항횟수_연도     145548 non-null  float64
 4   입항횟수_횟수     145548 non-null  float64
 5   구분          145548 non-null  object 
 6   입출          145548 non-null  object 
 7   총톤수         145548 non-null  object 
 8   입항일시        145548 non-null  object 
 9   출항일시        145548 non-null  object 
 10  수리일시        145510 non-null  object 
 11  항해구분        145548 non-null  object 
 12  계선장소_코드     145548 non-null  object 
 13  계선장소_숫자     145548 non-null  object 
 14  계선장소명       145548 non-null  object 
 15  선박용도        145548 non-null  object 
 16  순번          142894 non-null  object 
 17  시설사용횟수      142894 non-null  object 
 18  신고톤수        142894 non-null  object 
 19  지정

In [20]:
unique_values.isna().sum() # 계선장소, 예선, 도선, 차항지

항명               0
호출부호             0
선박명              0
입항횟수_연도          0
입항횟수_횟수          0
구분               0
입출               0
총톤수              0
입항일시             0
출항일시             0
수리일시            38
항해구분             0
계선장소_코드          0
계선장소_숫자          0
계선장소명            0
선박용도             0
순번            2654
시설사용횟수        2654
신고톤수          2654
지정시설_코드       2654
지정시설_번호       2654
지정시설명         2654
지정일시(FROM)    2654
지정일시(TO)      2654
사용목적명         2654
예보일시          2654
허가유무          2654
dtype: int64

In [21]:
unique_values.columns

Index(['항명', '호출부호', '선박명', '입항횟수_연도', '입항횟수_횟수', '구분', '입출', '총톤수', '입항일시',
       '출항일시', '수리일시', '항해구분', '계선장소_코드', '계선장소_숫자', '계선장소명', '선박용도', '순번',
       '시설사용횟수', '신고톤수', '지정시설_코드', '지정시설_번호', '지정시설명', '지정일시(FROM)',
       '지정일시(TO)', '사용목적명', '예보일시', '허가유무'],
      dtype='object')

In [22]:
unique_values

,항명,호출부호,선박명,입항횟수_연도,입항횟수_횟수,구분,입출,총톤수,입항일시,출항일시,...,시설사용횟수,신고톤수,지정시설_코드,지정시설_번호,지정시설명,지정일시(FROM),지정일시(TO),사용목적명,예보일시,허가유무
0,울산,3EFF5,EVER CHARM,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-06-16 20:05,...,001,17.933,MQM,01,현대미포의장안벽 01,2020-06-16 13:40,2020-06-16 13:50,기타,2020-06-17,허가완료
1,울산,3EFF5,EVER CHARM,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-06-16 20:05,...,002,17.933,WAB,02,(구)벙커링전용정박지 E2,2020-06-16 14:52,2020-06-16 20:05,급유,2020-06-17,허가완료
2,울산,3ETC7,EVER CHEER,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-07-07 19:42,...,001,17.933,MQM,04,현대미포의장안벽 04,2020-07-07 13:40,2020-07-07 13:50,기타,2020-07-09,허가완료
3,울산,3ETC7,EVER CHEER,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-07-07 19:42,...,002,17.933,WAB,02,(구)벙커링전용정박지 E2,2020-07-07 14:48,2020-07-07 19:42,급유,2020-07-09,허가완료
4,울산,3FAW9,ESTEEM HOUSTON,2020.0,1.0,최종,출항,"29,651",1950-01-01 00:00,2020-06-07 15:36,...,001,29.651,MQM,03,현대미포의장안벽 03,2020-06-05 16:20,2020-06-05 16:30,접안대기,2020-06-08,허가완료
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149800,울산,130064,에이원,2022.0,114.0,최종,출항,431,2022-12-31 09:45,2022-12-31 16:15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149801,울산,HOXG,SPRING NOTE,2022.0,31.0,최종,출항,"2,579",2022-12-31 10:05,2022-12-31 20:10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149802,울산,200013,유성,2022.0,142.0,최종,출항,"1,998",2022-12-31 15:55,2022-12-31 23:15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149803,울산,066597,한성8호,2022.0,22.0,최종,출항,164,2022-12-31 17:55,2022-12-31 20:40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3. 대기시간, 서비스시간 컬럼생성(Entry > Anchor > Berth > Departure)

In [23]:
temp = unique_values.copy()

In [24]:
# 데이터 타입으로 변경 , 잘못된 값 제거(6311 인덱스)
temp['지정일시(TO)'] = pd.to_datetime(temp['지정일시(TO)'], errors='coerce')
temp['지정일시(FROM)'] = pd.to_datetime(temp['지정일시(FROM)'], errors='coerce')

In [25]:
# 접안 대기시간,출항 대기시간 계산
temp['접안_대기시간'] = pd.NaT
temp['출항_대기시간'] = pd.NaT

temp.loc[temp['사용목적명'] == '접안대기', '접안_대기시간'] = temp['지정일시(TO)'] - temp['지정일시(FROM)']
temp.loc[temp['사용목적명'] == '출항대기', '출항_대기시간'] = temp['지정일시(TO)'] - temp['지정일시(FROM)']

In [26]:
temp.head()

,항명,호출부호,선박명,입항횟수_연도,입항횟수_횟수,구분,입출,총톤수,입항일시,출항일시,...,지정시설_코드,지정시설_번호,지정시설명,지정일시(FROM),지정일시(TO),사용목적명,예보일시,허가유무,접안_대기시간,출항_대기시간
0,울산,3EFF5,EVER CHARM,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-06-16 20:05,...,MQM,01,현대미포의장안벽 01,2020-06-16 13:40:00,2020-06-16 13:50:00,기타,2020-06-17,허가완료,NaT,NaT
1,울산,3EFF5,EVER CHARM,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-06-16 20:05,...,WAB,02,(구)벙커링전용정박지 E2,2020-06-16 14:52:00,2020-06-16 20:05:00,급유,2020-06-17,허가완료,NaT,NaT
2,울산,3ETC7,EVER CHEER,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-07-07 19:42,...,MQM,04,현대미포의장안벽 04,2020-07-07 13:40:00,2020-07-07 13:50:00,기타,2020-07-09,허가완료,NaT,NaT
3,울산,3ETC7,EVER CHEER,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-07-07 19:42,...,WAB,02,(구)벙커링전용정박지 E2,2020-07-07 14:48:00,2020-07-07 19:42:00,급유,2020-07-09,허가완료,NaT,NaT
4,울산,3FAW9,ESTEEM HOUSTON,2020.0,1.0,최종,출항,"29,651",1950-01-01 00:00,2020-06-07 15:36,...,MQM,03,현대미포의장안벽 03,2020-06-05 16:20:00,2020-06-05 16:30:00,접안대기,2020-06-08,허가완료,0 days 00:10:00,NaT


#### Service_Time_End 생성

- 출항대기시간 null 값인 경우에는, Departure 값이 '출항일시' 값
- 사용목적명이 '출항대기'일 경우에 Departure 값이 '지정일시(FROM)' 값으로

In [27]:
import numpy as np

# 'Service_Time_End' 열 생성
temp['Service_Time_End'] = np.nan

# '사용목적명'이 '출항대기'인 경우 'Service_Time_End' 값 할당
temp.loc[temp['사용목적명'] == '출항대기', 'Service_Time_End'] = temp['지정일시(FROM)']

# '출항_대기시간'이 null인 경우 'Service_Time_End' 값 할당
temp.loc[temp['출항_대기시간'].isnull(),'Service_Time_End'] = temp['출항일시']

In [28]:
temp.head(15)

,항명,호출부호,선박명,입항횟수_연도,입항횟수_횟수,구분,입출,총톤수,입항일시,출항일시,...,지정시설_번호,지정시설명,지정일시(FROM),지정일시(TO),사용목적명,예보일시,허가유무,접안_대기시간,출항_대기시간,Service_Time_End
0,울산,3EFF5,EVER CHARM,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-06-16 20:05,...,01,현대미포의장안벽 01,2020-06-16 13:40:00,2020-06-16 13:50:00,기타,2020-06-17,허가완료,NaT,NaT,2020-06-16 20:05
1,울산,3EFF5,EVER CHARM,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-06-16 20:05,...,02,(구)벙커링전용정박지 E2,2020-06-16 14:52:00,2020-06-16 20:05:00,급유,2020-06-17,허가완료,NaT,NaT,2020-06-16 20:05
2,울산,3ETC7,EVER CHEER,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-07-07 19:42,...,04,현대미포의장안벽 04,2020-07-07 13:40:00,2020-07-07 13:50:00,기타,2020-07-09,허가완료,NaT,NaT,2020-07-07 19:42
3,울산,3ETC7,EVER CHEER,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-07-07 19:42,...,02,(구)벙커링전용정박지 E2,2020-07-07 14:48:00,2020-07-07 19:42:00,급유,2020-07-09,허가완료,NaT,NaT,2020-07-07 19:42
4,울산,3FAW9,ESTEEM HOUSTON,2020.0,1.0,최종,출항,"29,651",1950-01-01 00:00,2020-06-07 15:36,...,03,현대미포의장안벽 03,2020-06-05 16:20:00,2020-06-05 16:30:00,접안대기,2020-06-08,허가완료,0 days 00:10:00,NaT,2020-06-07 15:36
5,울산,3FAW9,ESTEEM HOUSTON,2020.0,1.0,최종,출항,"29,651",1950-01-01 00:00,2020-06-07 15:36,...,02,정박지-E2,2020-06-05 18:00:00,2020-06-07 15:36:00,급유,2020-06-08,허가완료,NaT,NaT,2020-06-07 15:36
6,울산,3FKA6,POS HOCHIMINH,2020.0,1.0,최종,출항,"18,085",1950-01-01 00:00,2020-01-10 15:00,...,04,현대미포의장안벽 04,2020-01-10 15:05:00,2020-01-10 15:10:00,기타,2020-01-11,허가완료,NaT,NaT,2020-01-10 15:00
7,울산,3FWO,SOLAR KATHERINE,2020.0,1.0,최종,출항,"29,591",1950-01-01 00:00,2020-06-19 01:36,...,04,현대미포의장안벽 04,2020-06-18 15:55:00,2020-06-18 16:00:00,기타,2020-06-19,허가완료,NaT,NaT,2020-06-19 01:36
8,울산,3FWO,SOLAR KATHERINE,2020.0,1.0,최종,출항,"29,591",1950-01-01 00:00,2020-06-19 01:36,...,02,(구)벙커링전용정박지 E2,2020-06-18 17:10:00,2020-06-19 01:36:00,급유,2020-06-19,허가완료,NaT,NaT,2020-06-19 01:36
9,울산,3FXI8,ESTEEM ENERGY,2020.0,1.0,최종,출항,"29,651",1950-01-01 00:00,2020-08-01 07:24,...,03,현대미포의장안벽 03,2020-07-30 17:25:00,2020-07-30 17:35:00,기타,2020-08-03,허가완료,NaT,NaT,2020-08-01 07:24


#### Anchor = 접안대기

In [29]:
temp['Anchor'] = np.nan

# '사용목적명'이 '접안대기'인 경우 'Anchor' 값 할당
temp.loc[temp['사용목적명'] == '접안대기', 'Anchor'] = temp['지정일시(TO)']

In [30]:
temp['Service_Time_Start'] = np.where(temp['Anchor'].isnull(), temp['입항일시'], temp['Anchor'])

In [31]:
# 문자열을 datetime으로 변환
temp['Service_Time_End'] = pd.to_datetime(temp['Service_Time_End'])
temp['Service_Time_Start'] = pd.to_datetime(temp['Service_Time_Start'])
temp['Service_Time'] = temp['Service_Time_End'] - temp['Service_Time_Start']

In [32]:
temp

,항명,호출부호,선박명,입항횟수_연도,입항횟수_횟수,구분,입출,총톤수,입항일시,출항일시,...,지정일시(TO),사용목적명,예보일시,허가유무,접안_대기시간,출항_대기시간,Service_Time_End,Anchor,Service_Time_Start,Service_Time
0,울산,3EFF5,EVER CHARM,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-06-16 20:05,...,2020-06-16 13:50:00,기타,2020-06-17,허가완료,NaT,NaN,2020-06-16 20:05:00,NaN,1950-01-01 00:00:00,25734 days 20:05:00
1,울산,3EFF5,EVER CHARM,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-06-16 20:05,...,2020-06-16 20:05:00,급유,2020-06-17,허가완료,NaT,NaN,2020-06-16 20:05:00,NaN,1950-01-01 00:00:00,25734 days 20:05:00
2,울산,3ETC7,EVER CHEER,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-07-07 19:42,...,2020-07-07 13:50:00,기타,2020-07-09,허가완료,NaT,NaN,2020-07-07 19:42:00,NaN,1950-01-01 00:00:00,25755 days 19:42:00
3,울산,3ETC7,EVER CHEER,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-07-07 19:42,...,2020-07-07 19:42:00,급유,2020-07-09,허가완료,NaT,NaN,2020-07-07 19:42:00,NaN,1950-01-01 00:00:00,25755 days 19:42:00
4,울산,3FAW9,ESTEEM HOUSTON,2020.0,1.0,최종,출항,"29,651",1950-01-01 00:00,2020-06-07 15:36,...,2020-06-05 16:30:00,접안대기,2020-06-08,허가완료,0 days 00:10:00,NaN,2020-06-07 15:36:00,2020-06-05 16:30:00,2020-06-05 16:30:00,1 days 23:06:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149800,울산,130064,에이원,2022.0,114.0,최종,출항,431,2022-12-31 09:45,2022-12-31 16:15,...,NaT,NaN,NaN,NaN,NaN,NaN,2022-12-31 16:15:00,NaN,2022-12-31 09:45:00,0 days 06:30:00
149801,울산,HOXG,SPRING NOTE,2022.0,31.0,최종,출항,"2,579",2022-12-31 10:05,2022-12-31 20:10,...,NaT,NaN,NaN,NaN,NaN,NaN,2022-12-31 20:10:00,NaN,2022-12-31 10:05:00,0 days 10:05:00
149802,울산,200013,유성,2022.0,142.0,최종,출항,"1,998",2022-12-31 15:55,2022-12-31 23:15,...,NaT,NaN,NaN,NaN,NaN,NaN,2022-12-31 23:15:00,NaN,2022-12-31 15:55:00,0 days 07:20:00
149803,울산,066597,한성8호,2022.0,22.0,최종,출항,164,2022-12-31 17:55,2022-12-31 20:40,...,NaT,NaN,NaN,NaN,NaN,NaN,2022-12-31 20:40:00,NaN,2022-12-31 17:55:00,0 days 02:45:00


### 서비스타임,대기율, 용도별 대기율

In [33]:
waiting = temp.copy()

In [34]:
waiting['접안_대기시간'].fillna(0, inplace = True)

In [35]:
# import pandas as pd

# # 데이터프레임에서 '접안_대기시간'과 'Service_Time' 컬럼을 timedelta 형식으로 변환
waiting['접안_대기시간'] = pd.to_timedelta(waiting['접안_대기시간'])
waiting['Service_Time'] = pd.to_timedelta(waiting['Service_Time'])

# timedelta 값에서 일(day)로 변환
waiting['접안_대기시간_NumDays'] = waiting['접안_대기시간'].dt.days
waiting['Service_Time_NumDays'] = waiting['Service_Time'].dt.days

In [36]:
# '접안_대기시간' 컬럼에서 'days', 'hours', 'minutes'를 분리하여 새로운 열로 추가
waiting['접안_대기시간_일'] = waiting['접안_대기시간'].dt.days
waiting['접안_대기시간_시간'] = waiting['접안_대기시간'].dt.components.hours
waiting['접안_대기시간_분'] = waiting['접안_대기시간'].dt.components.minutes
# 'Service_Time' 컬럼에서 'days', 'hours', 'minutes'를 분리하여 새로운 열로 추가
waiting['Service_Time_일'] = waiting['Service_Time'].dt.days
waiting['Service_Time_시간'] = waiting['Service_Time'].dt.components.hours
waiting['Service_Time_분'] = waiting['Service_Time'].dt.components.minutes

# 'Service_Time'을 분으로 변환하여 'Service_Time_분' 열에 추가
waiting['Service_Time_분'] = waiting['Service_Time_일'] * 24 * 60 + waiting['Service_Time_시간'] * 60 + waiting['Service_Time_분']

# '접안_대기시간'을 분으로 변환하여 '접안_대기시간_분' 열에 추가
waiting['접안_대기시간_분'] = waiting['접안_대기시간_일'] * 24 * 60 + waiting['접안_대기시간_시간'] * 60 + waiting['접안_대기시간_분']

In [37]:
waiting['대기율'] = waiting['접안_대기시간_분'] / waiting['Service_Time_분']

In [38]:
waiting['대기율'].describe()

count    145523.0
mean          inf
std           NaN
min        -346.0
25%           0.0
50%           0.0
75%           0.0
max           inf
Name: 대기율, dtype: float64

In [39]:
waiting['대기율'].fillna(0, inplace = True)

In [40]:
waiting['대기율'].mean()

inf

In [41]:
q99 = np.percentile(waiting['대기율'], 99)  
outliers = waiting[waiting['대기율'] > q99]

# 이상치 제외한 데이터 생성
filtered_data = waiting[waiting['대기율'] <= q99]  # 상위 1%의 이상치를 제외한 데이터

In [42]:
filtered_data['대기율'].mean()

0.1483057062538403

In [43]:
waiting.columns

Index(['항명', '호출부호', '선박명', '입항횟수_연도', '입항횟수_횟수', '구분', '입출', '총톤수', '입항일시',
       '출항일시', '수리일시', '항해구분', '계선장소_코드', '계선장소_숫자', '계선장소명', '선박용도', '순번',
       '시설사용횟수', '신고톤수', '지정시설_코드', '지정시설_번호', '지정시설명', '지정일시(FROM)',
       '지정일시(TO)', '사용목적명', '예보일시', '허가유무', '접안_대기시간', '출항_대기시간',
       'Service_Time_End', 'Anchor', 'Service_Time_Start', 'Service_Time',
       '접안_대기시간_NumDays', 'Service_Time_NumDays', '접안_대기시간_일', '접안_대기시간_시간',
       '접안_대기시간_분', 'Service_Time_일', 'Service_Time_시간', 'Service_Time_분',
       '대기율'],
      dtype='object')

In [44]:
waiting

,항명,호출부호,선박명,입항횟수_연도,입항횟수_횟수,구분,입출,총톤수,입항일시,출항일시,...,Service_Time,접안_대기시간_NumDays,Service_Time_NumDays,접안_대기시간_일,접안_대기시간_시간,접안_대기시간_분,Service_Time_일,Service_Time_시간,Service_Time_분,대기율
0,울산,3EFF5,EVER CHARM,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-06-16 20:05,...,25734 days 20:05:00,0,25734,0,0,0,25734,20,37058165,0.000000
1,울산,3EFF5,EVER CHARM,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-06-16 20:05,...,25734 days 20:05:00,0,25734,0,0,0,25734,20,37058165,0.000000
2,울산,3ETC7,EVER CHEER,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-07-07 19:42,...,25755 days 19:42:00,0,25755,0,0,0,25755,19,37088382,0.000000
3,울산,3ETC7,EVER CHEER,2020.0,1.0,최종,출항,"17,933",1950-01-01 00:00,2020-07-07 19:42,...,25755 days 19:42:00,0,25755,0,0,0,25755,19,37088382,0.000000
4,울산,3FAW9,ESTEEM HOUSTON,2020.0,1.0,최종,출항,"29,651",1950-01-01 00:00,2020-06-07 15:36,...,1 days 23:06:00,0,1,0,0,10,1,23,2826,0.003539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149800,울산,130064,에이원,2022.0,114.0,최종,출항,431,2022-12-31 09:45,2022-12-31 16:15,...,0 days 06:30:00,0,0,0,0,0,0,6,390,0.000000
149801,울산,HOXG,SPRING NOTE,2022.0,31.0,최종,출항,"2,579",2022-12-31 10:05,2022-12-31 20:10,...,0 days 10:05:00,0,0,0,0,0,0,10,605,0.000000
149802,울산,200013,유성,2022.0,142.0,최종,출항,"1,998",2022-12-31 15:55,2022-12-31 23:15,...,0 days 07:20:00,0,0,0,0,0,0,7,440,0.000000
149803,울산,066597,한성8호,2022.0,22.0,최종,출항,164,2022-12-31 17:55,2022-12-31 20:40,...,0 days 02:45:00,0,0,0,0,0,0,2,165,0.000000


In [45]:
result = waiting.groupby('선박용도')['대기율'].mean().to_frame()
result = result.sort_values(by='대기율', ascending=False).reset_index()

In [46]:
result.sort_values(by = '대기율', ascending = False)

,선박용도,대기율
0,여객선,0.615733
1,철강재 운반선,0.221373
2,세미(혼재)컨테이너선,0.167599
3,신조선,0.035794
4,시멘트운반선,0.019899
5,압항 예선,0.015639
6,풀컨테이너선,0.004212
7,기타 예선,0.000671
16,관공선,0.000000
22,광석운반선,0.000000


In [47]:
waiting.to_csv(directory + '/울산_전처리ver1.csv', encoding = 'utf-8-sig', index = False)

In [48]:
waiting.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145548 entries, 0 to 149804
Data columns (total 42 columns):
 #   Column                Non-Null Count   Dtype          
---  ------                --------------   -----          
 0   항명                    145548 non-null  object         
 1   호출부호                  145548 non-null  object         
 2   선박명                   145548 non-null  object         
 3   입항횟수_연도               145548 non-null  float64        
 4   입항횟수_횟수               145548 non-null  float64        
 5   구분                    145548 non-null  object         
 6   입출                    145548 non-null  object         
 7   총톤수                   145548 non-null  object         
 8   입항일시                  145548 non-null  object         
 9   출항일시                  145548 non-null  object         
 10  수리일시                  145510 non-null  object         
 11  항해구분                  145548 non-null  object         
 12  계선장소_코드               145548 non-null  objec

#### 이상치 제거(상하위 1%)

In [49]:
# '대기율' 열에서 상하위 1%에 해당하는 값 구하기
lower_bound = waiting['대기율'].quantile(0.01)
upper_bound = waiting['대기율'].quantile(0.99)

# 상하위 1%에 해당하는 이상치 제거
filtered_waiting = waiting[(waiting['대기율'] >= lower_bound) & (waiting['대기율'] <= upper_bound)]

In [50]:
filtered_waiting.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143915 entries, 0 to 149804
Data columns (total 42 columns):
 #   Column                Non-Null Count   Dtype          
---  ------                --------------   -----          
 0   항명                    143915 non-null  object         
 1   호출부호                  143915 non-null  object         
 2   선박명                   143915 non-null  object         
 3   입항횟수_연도               143915 non-null  float64        
 4   입항횟수_횟수               143915 non-null  float64        
 5   구분                    143915 non-null  object         
 6   입출                    143915 non-null  object         
 7   총톤수                   143915 non-null  object         
 8   입항일시                  143915 non-null  object         
 9   출항일시                  143915 non-null  object         
 10  수리일시                  143877 non-null  object         
 11  항해구분                  143915 non-null  object         
 12  계선장소_코드               143915 non-null  objec

In [51]:
filtered_waiting['대기율'].describe()

count    143915.000000
mean          0.155871
std           0.582932
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           5.291045
Name: 대기율, dtype: float64

In [52]:
filtered_waiting.to_csv(directory + '/울산_전처리_202122.csv', encoding = 'utf-8-sig', index = False)